# Incidence Matrices for quasi-cords

## modules

In [1]:
import numpy as np
import plotly.graph_objects as go

## Classes and functions

### Square class, Side class, Segment class

In [2]:
LABELS = ['T','L','B','R']

class Square:
    '''A square obtained by cut the disk with 4 points T,L,B,R along the arcs a_T, a_L, a_B, a_R'''
    
    def __init__(self, code):
        '''
        Arguments
        ---------
        code : tuple
            The 4-tuple corresponding to the quasi-cord.
        '''
        self.code = code                      # tuple: code = (\d, \d, \d, \d)
        self.sides = self.get_sides()         # dict: sides = {label: Side(label, num),...}
        self.segments = self.get_segments()   # list: segments = [Segment,...]
   
    def get_sides(self):
        sides_dict = {}
        for i in range(len(LABELS)):
            sides_dict[LABELS[i]] = Side(LABELS[i], self.code[i])
        return sides_dict

    def get_segments(self):
        segs_list = []
        
        # create the top-left segments 
        tl = self.sides['T'].number + self.sides['L'].number
        br = self.sides['B'].number + self.sides['R'].number  #TODO: tl >= br でなかったらエラーを吐くようにする。
        diff = tl - br
        for i in range(diff):
            a_tlseg = Segment(self.sides['T'].points[i], self.sides['L'].points[i])
            segs_list.append(a_tlseg)
        
        # the remaining
        remaining = []
        TL_points = list(reversed(self.sides['L'].points[diff:]))+self.sides['T'].points[diff:]
        BR_points = self.sides['B'].points+self.sides['R'].points

        for i in range(len(TL_points)):
            a_seg = Segment(TL_points[i], BR_points[i])
            segs_list.append(a_seg)
        return segs_list
    
#---------------------------------------------------------
class Side:
    '''A side of the square'''
    
    def __init__(self, label, number):
        '''
        Arguments
        ---------
        label : string
            One of the letters in {T,L,B,R}.
        number : int
            The number coressponding to the points on the side of the label.
        '''
        self.label = label
        self.number = number
        self.points = self.get_points()
        
    def get_points(self):
        points = [self.label+str(k)+'+' for k in range(self.number)]
        points += [label[:-1]+'-' for label in reversed(points)]
        if self.label in ['L','R']:
            idx = int(len(points)/2)
            points.insert(idx, self.label)
        return points

class Segment:
    '''A segment on the square connecting two points on the sides'''
    
    def __init__(self, point01, point02):
        '''
        Arguments
        ---------
        point01, point02 : string
            The labels of the two points on the sides.
        '''
        self.ends = (point01, point02)
        
    def show(self):
        return self.ends

### get_matrix function

In [3]:
def get_matrix(sqr):
    variables = []
    for label in LABELS:
        num = sqr.sides[label].number
        if label in ['L','R']:
            num += 1
        variables.append(sqr.sides[label].points[:num])
    variables = [v.replace('+','') for v in sum(variables, [])] # flatten and remove '+'
#     print(variables)
    #---
    s = len(sqr.segments)
    M = np.zeros((s,s+1), dtype=int)
    raw = 0
    for a_seg in sqr.segments:
        i = 0
#         print("{} --> {}-{}=0".format(a_seg.ends, a_seg.ends[0], a_seg.ends[1]))
        for end_point in a_seg.ends:
            if end_point[-1] in ['+','-']:
                end_point = end_point[:-1]  # remove '+' and '-'
            M[raw][variables.index(end_point)] = (-1)**i
            i += 1
        raw += 1
#     else: print('\n')

    tl = sqr.sides['T'].number + sqr.sides['L'].number
    C = np.roll(np.eye(1, s+1, dtype=int), shift=tl, axis=1)
    M = np.r_[M, C]
    return M

### show_graphic function

In [218]:
def show_graphic(sqr, size=5, figsize=(600,600)):
    fig = go.Figure()
    SIZE = size
    
    #--- Points
    moves = {'T': (1, SIZE*1j), 'L': (-1j, SIZE*(-1)), 'B': (1, SIZE*1j*(-1)), 'R': (1j, SIZE)}
    all_points_dict = {}
    for side in sqr.sides.values():
#         print("side {} --> {} / {}".format(side.label, side.number, side.points))
        num = side.number
        delta = SIZE/(num+1)
        #---
        complex_list = list(reversed([(-1)*delta*(k+1) for k in range(num)]))
        if side.label in ['L', 'R']:
            complex_list.append(0)
        complex_list += [delta*(k+1) for k in range(num)]
        complex_list = [v*moves[side.label][0]+moves[side.label][1] for v in complex_list]
        #---
#         points_coords_list = zip(side.points, complex_list)
# #         print(side.points, complex_list, points_coords_list)
#         for point_label, z in points_coords_list:
#             fig.add_trace(go.Scatter(
#                 x = [z.real],
#                 y = [z.imag],
#                 mode="lines+markers+text",
#                 legendgroup = side.label,
#                 text = point_label,
#             ))
#             all_points_dict[point_label] = z
        fig.add_trace(go.Scatter(
            x = [z.real for z in complex_list],
            y = [z.imag for z in complex_list],
            mode="lines+markers+text",
        ))
        for k in range(len(side.points)):
            all_points_dict[side.points[k]] = complex_list[k]
#        
#     print(all_points_dict)
     
    #--- Frame
    frame = go.layout.Shape(
                type="rect",
                x0=SIZE, y0=SIZE, x1=(-1)*SIZE, y1=(-1)*SIZE,
                line=dict(color="RoyalBlue",),
            )
    #--- Segments
    my_shapes = [frame]
    for seg in sqr.segments:
        pnt0 = all_points_dict[seg.ends[0]]
        pnt1 = all_points_dict[seg.ends[1]]
#         print("{},{}".format(seg.ends, (pnt0, pnt1)))
        my_shapes.append(
            go.layout.Shape(
                type="line",
                x0=pnt0.real, y0=pnt0.imag,
                x1=pnt1.real, y1=pnt1.imag,
                line=dict(
                    color="LightSeaGreen",
                    width=3,
                ),
            )
        )
    else: fig.update_layout(shapes=my_shapes)
     
    #--- Update axes properties
    rng = SIZE*1.2
    fig.update_xaxes(
        range=[(-1)*rng, rng], 
        showticklabels=False,
        showgrid=False,
        zeroline=False,
    )
    fig.update_yaxes(
        range=[(-1)*rng, rng], 
        showticklabels=False,
        showgrid=False,
        zeroline=False,
    )
    #--- Set figure size and background color
    fig.update_layout(width=figsize[0], height=figsize[1], plot_bgcolor="white", showlegend=False)
    fig.show()

## Create an instance of Square class.

In [238]:
def square_random_generator():
    code = [np.random.randint(1,10) for k in range(4)]
    tl = sum(code[:2])
    br = sum(code[-2:])
    if tl < br:
        code = reversed(code)
    return Square(tuple(code))

#sqr = square_random_generator()
sqr = Square((0,1,0,0)) #0,3,0,3)) #8,4,6,2)) #3,2,1,1)) #(3,4,5,2)) #(3,7,4,2)) #(1,3,1,1)) #(3,4,2,1)) #

print(sqr.code)

IndexError: list index out of range

### Parts of a square

End points on the sides

In [6]:
for side in sqr.sides.values():
    print("side {} --> {}".format(side.label, side.points))

side T --> ['T0+', 'T1+', 'T2+', 'T3+', 'T4+', 'T5+', 'T6+', 'T7+', 'T8+', 'T8-', 'T7-', 'T6-', 'T5-', 'T4-', 'T3-', 'T2-', 'T1-', 'T0-']
side L --> ['L0+', 'L1+', 'L', 'L1-', 'L0-']
side B --> ['B0+', 'B1+', 'B2+', 'B3+', 'B4+', 'B5+', 'B5-', 'B4-', 'B3-', 'B2-', 'B1-', 'B0-']
side R --> ['R0+', 'R1+', 'R2+', 'R3+', 'R4+', 'R', 'R4-', 'R3-', 'R2-', 'R1-', 'R0-']


Segments

In [7]:
all_segs = [seg.ends for seg in sqr.segments]
print(all_segs)

[('L0-', 'B0+'), ('L1-', 'B1+'), ('L', 'B2+'), ('L1+', 'B3+'), ('L0+', 'B4+'), ('T0+', 'B5+'), ('T1+', 'B5-'), ('T2+', 'B4-'), ('T3+', 'B3-'), ('T4+', 'B2-'), ('T5+', 'B1-'), ('T6+', 'B0-'), ('T7+', 'R0+'), ('T8+', 'R1+'), ('T8-', 'R2+'), ('T7-', 'R3+'), ('T6-', 'R4+'), ('T5-', 'R'), ('T4-', 'R4-'), ('T3-', 'R3-'), ('T2-', 'R2-'), ('T1-', 'R1-'), ('T0-', 'R0-')]


### Graphic

In [237]:
show_graphic(sqr, size=10, figsize=(500,500))

### Incidence Matrix

Create the coefficient matrix of the equations coresponding to the segments

In [234]:
M = get_matrix(sqr)
print("{} --det--> {}".format(M, np.linalg.det(M)))

[[ 0  1 -1  0]
 [ 1  0 -1  0]
 [ 1  0  0 -1]
 [ 0  1  0  0]] --det--> -1.0


### Connecting $\pm 1$'s on the matrix

In [235]:
fig = go.Figure()
figsize = (500, 500)
SIZE = 10
s = len(sqr.segments)
delta = SIZE/(s+1)

def trans(pair):
    return (pair[1] - pair[0]*1j)*delta +(3/2+1/2*1j)*delta + SIZE*1j

#--- Points ---
corners = dict(points=[], marker=dict(size=5.0, color="Red"),)
others = dict(points=[], marker=dict(size=2.0, color="RoyalBlue"),)

entry = np.nditer(M, flags=['multi_index'])
while not entry.finished:
    if entry[0]==0:
        others['points'].append(entry.multi_index)
    else:
        corners['points'].append(entry.multi_index)
    entry.iternext()

for pdict in [others, corners]:
    z_list = [trans(pair) for pair in pdict['points']]
    fig.add_trace(go.Scatter(
        x = [z.real for z in z_list],
        y = [z.imag for z in z_list],
        mode="markers",
        marker=pdict['marker'],
    ))

#--- Lines ---
lines = []
for k in range(s):
    for i in range(2):  # 0: horizontal lines, 1: vertical lines
        z = [trans(pair) for pair in corners['points'] if pair[i] == k]
        if len(z) == 2:
            lines.append(
                go.layout.Shape(
                    type="line",
                    x0=z[0].real, y0=z[0].imag,
                    x1=z[1].real, y1=z[1].imag,
                    line=dict(color="LightSeaGreen", width=1,),
                ))
               
#--- Update axes properties
options = dict(
    range=[0, SIZE+2*delta], 
    showticklabels=False,
    showgrid=False,
    zeroline=False,
)
fig.update_xaxes(options)
fig.update_yaxes(options)
#--- Set figure size and background color
fig.update_layout(
    width=figsize[0], height=figsize[1],
#     plot_bgcolor="white", 
    showlegend=False,
    shapes=lines
)
fig.show()

## Experiments

In [188]:
mdict = dict(points=[0,1])
print(mdict)

{'points': [0, 1]}


### Cofactor expansion 

In [36]:
def cofexp_raw(sqr_matrix, raw_num=0):
    expansion = []
    raw = sqr_matrix[raw_num]
    for j in range(raw.size):
        if raw[j] != 0:
            shrinked = np.delete(np.delete(sqr_matrix, raw_num, 0),j,1)
            sign = (-1)**(raw_num+j)
            expansion.append([sign*raw[j], shrinked])
    return expansion

__予想1:__ 接続行列のどの行(最終行を除く)で余因子展開しても、
全体が正則なら、必ず行列式 $\pm 1$ のものが一つだけ。

これが正しいなら、正則な接続行列の行列式は $\pm 1$ であるとも言える。

__予想2:__ 同様の余因子展開について、非正則ならすべて行列式 $0$.

こちらの予想は反例があった。ex) Square(9,7,9,7), Square(8,4,6,2)

In [214]:
#sqr = square_random_generator(); 
#sqr = Square((0,3,0,3))
#M = get_matrix(sqr)

print("{} --det--> {}\n".format(sqr.code, np.linalg.det(M)))
# print("{}\n".format(M))

for rn in range(len(sqr.segments)):
    expn = cofexp_raw(M, raw_num=rn)
    values = []
    for pair in expn:
        m = pair[1]
        values.append(int(np.linalg.det(m)))
    else: print("raw {} --raw_exp--> {}".format(rn, values))

(2, 6, 3, 2) --det--> -1.0

raw 0 --raw_exp--> [0, 1]
raw 1 --raw_exp--> [0, 1]
raw 2 --raw_exp--> [1, 0]
raw 3 --raw_exp--> [0, 1]
raw 4 --raw_exp--> [0, 1]
raw 5 --raw_exp--> [1, 0]
raw 6 --raw_exp--> [0, -1]
raw 7 --raw_exp--> [1, 0]
raw 8 --raw_exp--> [1, 0]
raw 9 --raw_exp--> [0, -1]
raw 10 --raw_exp--> [0, -1]
raw 11 --raw_exp--> [0, -1]
raw 12 --raw_exp--> [1, 0]
raw 13 --raw_exp--> [1, 0]


__予想2(修正):__ 同様の余因子展開について、非正則なら、どの行でも 2つの行列式を足すと $0\mod 2$.


In [74]:
def recursive_cofexp_raw(coeff_matrix_pair, count=2):
    if count > 0:
        count += -1
        sqr_matrix = coeff_matrix_pair[1]
#         print(sqr_matrix)
        coeff_matrix_pair[1] = [recursive_cofexp_raw(pair, count) for pair in cofexp_raw(sqr_matrix)]
    return coeff_matrix_pair

In [163]:
expansion = recursive_cofexp_raw([1,M], count=5)
# print(expansion)

def show_expansion(expn, depth=0):
    for v in expn:
        if isinstance(v, list):
            show_expansion(v, depth+1)
        else:
            if isinstance(v, np.ndarray):
                det = np.linalg.det(v)
                if det != 0:
                    print("{} --> {}".format(v, np.linalg.det(v)))

show_expansion(expansion, depth=3)            

[[ 0  0  0  0  0 -1  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0 -1  0  0  0]
 [ 0  0  0  1  0  0  0 -1  0  0]
 [ 0  0  0  0  1  0  0  0 -1  0]
 [ 0  0  0  1  0  0  0  0  0 -1]
 [ 0  0  1  0  0  0  0  0 -1  0]
 [ 0  1  0  0  0  0  0 -1  0  0]
 [ 1  0  0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  1  0  0  0  0  0]] --> 1.0
[[ 0  1  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0  0  0  0  0]
 [ 0  0  0  1  0  0 -1  0  0  0]
 [ 0  0  0  0  1  0  0 -1  0  0]
 [ 0  0  0  0  0  1  0  0 -1  0]
 [ 0  0  0  0  1  0  0  0  0 -1]
 [ 0  0  0  1  0  0  0  0 -1  0]
 [ 0  0  1  0  0  0  0 -1  0  0]
 [ 1  0  0  0  0  0 -1  0  0  0]
 [ 0  0  0  0  0  1  0  0  0  0]] --> -1.0


### Elementary deformations

In [34]:
def elem_matrix(size=1, position=(1,1)):
    i,j = position
    P = np.eye(size, size, dtype=int)
    P[i][j] = 1
    return P
def P(n,ij):
    return elem_matrix(size=n, position=ij)

deform an incidence matrix by elementary matrices

In [35]:
s = len(sqr.segments)
N = M

num = 0
for side in sqr.sides.values():
    for k in range(side.number):
            N = N@P(s+1,(num+k+1, num+0))
            N = P(s+1,(num+0, num+k+1))@N
    num += side.number

print(N)

[[ 3  1  1 -2 -1 -2 -1  0  0]
 [ 1  1  0 -1 -1  0  0  0  0]
 [ 1  0  1 -1  0 -1  0  0  0]
 [ 2  0  1  2  1 -2 -3 -1  0]
 [ 0  0  0  1  1 -1 -1  0  0]
 [ 2  1  1  0  0  0 -1 -1 -1]
 [ 2  1  0  0  0  0 -1 -1 -1]
 [ 1  0  0  0  0  0 -1 -1  0]
 [ 0  0  0  1  0  1  0  0  0]]


### Others

In [ ]:
s = len(sqr.segments)
# N = M
N[s][s]=1
print("{} --det--> {}".format(N, np.linalg.det(N)))

## TODO

* [ ] 接続行列に対して、基本変形を自由にできるようにする。
* [x] show_graphic 関数で、なぜか最初の segment を描画してくれない。
* [x] 行列生成処理も何らかのクラスのメソッドにして、大量計算を実行できるようにしたい。
* [x] 接続行列の生成について。quasi-cord が $\tau + \lambda = \beta + \rho$ の場合しかうまく行ってない。